CNN with a Gated Attention mechanism (Conv-GAtt) Code for Horizon 1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    df['month'] = df.index.month
    df['weekofyear'] = df.index.isocalendar().week
    df['year'] = df.index.isocalendar().year
    df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    fft_values = fft(df[target_col].dropna().values)
    fft_real = np.real(fft_values)[:len(df[target_col])]
    fft_imag = np.imag(fft_values)[:len(df[target_col])]
    df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 3D
    return X_scaled, y, scaler

# ============================================
# 3. CNN + Gate Attention Model
# ============================================
def gated_attention_block(inputs, units):
    conv_output = layers.Conv1D(units, kernel_size=1, activation='relu')(inputs)
    attention_scores = layers.Dense(units, activation='sigmoid')(conv_output)
    gated_output = layers.Multiply()([conv_output, attention_scores])
    return gated_output

def build_cnn_gatt_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    inputs = tf.keras.Input(shape=input_shape)
    x = gated_attention_block(inputs, units=num_units)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(num_units // 2, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_cnn_gatt_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_cnn_gatt_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final Gate Attention CNN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 1.0       | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 1.0       | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 1.0       | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 1.0       | 58.87     | 0.237     | 99.91     | 32.34     |
| 7         | 1.0       | 63.83     | 0.1911    | 99.11     | 127.8     |
| 8         | 0.9919    | 17.24     | 0.4906    | 50.56     | 33.63     |
| 9         | 1.0       | 21.82     | 0.3011    | 51.99     | 115.7     |
| 10        | 1.0       | 63.61     | 0.2064    | 87.45     | 68.96     |
| 11        | 1.0       | 62.61     | 0.4329    | 50.58     | 87.37     |
| 12  


===== Evaluation =====
Accuracy: 0.9606
Report:
               precision    recall  f1-score   support

           0     1.0000    0.5714    0.7273        56
           1     0.9087    0.9958    0.9503       240
           2     0.9959    0.9877    0.9918       487
           3     0.8286    0.9667    0.8923        30

    accuracy                         0.9606       813
   macro avg     0.9333    0.8804    0.8904       813
weighted avg     0.9643    0.9606    0.9576       813

Confusion Matrix:
 [[ 32  24   0   0]
 [  0 239   1   0]
 [  0   0 481   6]
 [  0   0   1  29]]

===== Model Metrics =====
Training Time: 19.52s
Inference/sample: 0.000364s
Testing Time: 0.30s
Model Size: 0.08 MB
Trainable Params: 14,731
RAM Usage: 856.61 MB


CNN with a Gated Attention mechanism (Conv-GAtt) Code for Horizon 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h2'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1


    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 3D
    return X_scaled, y, scaler

# ============================================
# 3. CNN + Gate Attention Model
# ============================================
def gated_attention_block(inputs, units):
    conv_output = layers.Conv1D(units, kernel_size=1, activation='relu')(inputs)
    attention_scores = layers.Dense(units, activation='sigmoid')(conv_output)
    gated_output = layers.Multiply()([conv_output, attention_scores])
    return gated_output

def build_cnn_gatt_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    inputs = tf.keras.Input(shape=input_shape)
    x = gated_attention_block(inputs, units=num_units)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(num_units // 2, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_cnn_gatt_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_cnn_gatt_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\train_lasso_selected.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\test_lasso_selected.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final Gate Attention CNN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9959    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 1.0       | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9959    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 1.0       | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 0.9959    | 31.24     | 0.1594    | 50.22     | 125.1     |
| 7         | 1.0       | 18.36     | 0.1074    | 80.03     | 101.5     |
| 8         | 1.0       | 16.21     | 0.1161    | 73.32     | 65.8      |
| 9         | 1.0       | 62.6      | 0.3838    | 99.69     | 67.9      |
| 10        | 1.0       | 38.63     | 0.1642    | 90.1      | 41.26     |
| 11        | 1.0       | 61.04     | 0.3825    | 72.51     | 127.9     |
| 12  


===== Evaluation =====
Accuracy: 0.9680
Report:
               precision    recall  f1-score   support

           0     0.9091    0.4762    0.6250        42
           1     0.9102    0.9867    0.9469       226
           2     0.9980    0.9980    0.9980       509
           3     0.9730    1.0000    0.9863        36

    accuracy                         0.9680       813
   macro avg     0.9476    0.8652    0.8891       813
weighted avg     0.9679    0.9680    0.9640       813

Confusion Matrix:
 [[ 20  22   0   0]
 [  2 223   1   0]
 [  0   0 508   1]
 [  0   0   0  36]]

===== Model Metrics =====
Training Time: 11.60s
Inference/sample: 0.000287s
Testing Time: 0.23s
Model Size: 0.11 MB
Trainable Params: 21,104
RAM Usage: 1148.95 MB


CNN with a Gated Attention mechanism (Conv-GAtt) Code for Horizon 3

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h3'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 3D
    return X_scaled, y, scaler

# ============================================
# 3. CNN + Gate Attention Model
# ============================================
def gated_attention_block(inputs, units):
    conv_output = layers.Conv1D(units, kernel_size=1, activation='relu')(inputs)
    attention_scores = layers.Dense(units, activation='sigmoid')(conv_output)
    gated_output = layers.Multiply()([conv_output, attention_scores])
    return gated_output

def build_cnn_gatt_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    inputs = tf.keras.Input(shape=input_shape)
    x = gated_attention_block(inputs, units=num_units)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(num_units // 2, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_cnn_gatt_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_cnn_gatt_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H3_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H3_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final Gate Attention CNN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9919    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9959    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9919    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9919    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9959    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 0.9878    | 30.28     | 0.3338    | 71.98     | 61.01     |
| 7         | 0.9878    | 30.43     | 0.4113    | 71.94     | 59.91     |
| 8         | 1.0       | 16.21     | 0.1161    | 73.32     | 65.8      |
| 9         | 0.9919    | 62.6      | 0.3838    | 99.69     | 67.9      |
| 10        | 0.9837    | 38.63     | 0.1642    | 90.1      | 41.26     |
| 11        | 0.9919    | 61.04     | 0.3825    | 72.51     | 127.9     |
| 12  


===== Evaluation =====
Accuracy: 0.9508
Report:
               precision    recall  f1-score   support

           0     1.0000    0.4074    0.5789        27
           1     0.9256    0.9343    0.9299       213
           2     0.9866    0.9809    0.9837       524
           3     0.7424    1.0000    0.8522        49

    accuracy                         0.9508       813
   macro avg     0.9136    0.8306    0.8362       813
weighted avg     0.9563    0.9508    0.9483       813

Confusion Matrix:
 [[ 11  16   0   0]
 [  0 199   7   7]
 [  0   0 514  10]
 [  0   0   0  49]]

===== Model Metrics =====
Training Time: 22.99s
Inference/sample: 0.000347s
Testing Time: 0.28s
Model Size: 0.06 MB
Trainable Params: 7,704
RAM Usage: 1404.92 MB


CNN with a Gated Attention mechanism (Conv-GAtt) Code for Horizon 4

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h4'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 3D
    return X_scaled, y, scaler

# ============================================
# 3. CNN + Gate Attention Model
# ============================================
def gated_attention_block(inputs, units):
    conv_output = layers.Conv1D(units, kernel_size=1, activation='relu')(inputs)
    attention_scores = layers.Dense(units, activation='sigmoid')(conv_output)
    gated_output = layers.Multiply()([conv_output, attention_scores])
    return gated_output

def build_cnn_gatt_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    inputs = tf.keras.Input(shape=input_shape)
    x = gated_attention_block(inputs, units=num_units)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(num_units // 2, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_cnn_gatt_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_cnn_gatt_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H4_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H4_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final Gate Attention CNN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9919    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9919    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9919    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9837    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9837    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 0.9959    | 33.45     | 0.2494    | 99.66     | 127.5     |
| 7         | 0.9919    | 63.83     | 0.1911    | 99.11     | 127.8     |
| 8         | 0.9959    | 34.57     | 0.107     | 98.66     | 124.8     |
| 9         | 0.9959    | 16.43     | 0.3942    | 99.17     | 113.4     |
| 10        | 0.9959    | 16.05     | 0.4561    | 80.68     | 125.0     |
| 11        | 0.9959    | 18.27     | 0.3674    | 99.38     | 128.0     |
| 12  


===== Evaluation =====
Accuracy: 0.9557
Report:
               precision    recall  f1-score   support

           0     1.0000    0.3077    0.4706        13
           1     0.9579    0.9146    0.9357       199
           2     0.9852    0.9815    0.9833       541
           3     0.7500    1.0000    0.8571        60

    accuracy                         0.9557       813
   macro avg     0.9233    0.8009    0.8117       813
weighted avg     0.9614    0.9557    0.9542       813

Confusion Matrix:
 [[  4   8   0   1]
 [  0 182   8   9]
 [  0   0 531  10]
 [  0   0   0  60]]

===== Model Metrics =====
Training Time: 33.03s
Inference/sample: 0.000425s
Testing Time: 0.35s
Model Size: 0.05 MB
Trainable Params: 7,278
RAM Usage: 1657.35 MB
